 # GOLUB use case

In [1]:
library("evaluomeR") 
library("clusterAnalysis")
library("cancerclass")
library("dplyr")
library("caret")
library("MLmetrics")
library("ggplot2")
library("ggrepel")
library("reshape2")

options(scipen=10)
options(repr.plot.width=12, repr.plot.height=12)

Cargando paquete requerido: SummarizedExperiment

Cargando paquete requerido: MatrixGenerics

Cargando paquete requerido: matrixStats


Adjuntando el paquete: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles,

In [2]:
getRversion()
packageVersion("clusterAnalysis")

[1] '4.4.1'

[1] '0.0.0.9000'

# Table of contents
* [Dataset](#dataset)
    * [Top 200](#top)
* [Cest](#clest)
    * [RSKC](#rskc)
* [PCA](#pca)
* [Sensitivity](#sensitivity)
* [CER](#cer)

# Dataset <a class="anchor" id="dataset"></a>

In [21]:
data("golub")
dataset = golub
colnames(dataset)[colnames(dataset) == 'Case'] <- 'Description'
head(dataset)
dataset["Class"] = NULL
seed = 13606
set.seed(seed)

,Description,AFFX.HUMRGE.M10098_5_at,AFFX.HUMRGE.M10098_M_at,AFFX.HUMRGE.M10098_3_at,AFFX.M27830_5_at,D13639_at,D83735_at,D83920_at,D87433_at,D88270_at,⋯,X65965_s_at,X76223_s_at,AF000424_s_at,M21305_at,U57341_at,HG3576.HT3779_f_at,U01317_cds4_at,M15395_at,M34516_at,Class
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,B1,4.16250,3.98847,3.93090,3.82930,3.67274,2.00000,2.00000,2.0000,3.29336,⋯,2.83059,2.00000,3.49914,2.91960,2.30750,3.92536,2.07555,2.23553,3.82595,B
2,T1,2.78888,2.06070,3.18127,3.34537,3.52724,3.04218,3.35545,2.0000,2.29667,⋯,3.40926,3.96624,3.71533,2.00000,2.69810,3.25479,2.00000,3.19535,3.46195,T
3,T2,3.75351,3.51481,3.56443,3.52179,2.00000,2.89321,2.31175,2.0000,2.76268,⋯,2.52763,4.20412,2.85003,3.38364,2.64345,2.00000,2.23553,2.50379,2.00000,T
4,B2,3.68574,3.36040,3.40976,3.48544,2.00432,2.00000,2.00000,2.0000,3.68851,⋯,2.43457,2.00000,2.00000,2.79029,2.98632,3.42911,2.59660,2.37658,3.14364,B
5,B3,3.10857,3.43632,2.49969,3.05308,3.10585,2.00000,2.30750,2.4014,3.52930,⋯,2.56820,2.00000,2.73640,2.45637,2.22272,3.37014,2.09342,3.03383,3.36399,B
6,T3,3.61784,3.43823,3.38093,3.59173,2.00000,2.00000,3.18639,2.0000,2.00000,⋯,2.50651,4.13117,3.25888,2.00000,2.07918,2.00000,2.00000,3.01536,2.00000,T


In [22]:
level_mapping <- c("B" = 1, "T" = 2, "M" = 3)
map_strings_to_numbers <- function(strings) {
  factorized <- factor(strings, levels = names(level_mapping))
  as.numeric(factorized)
}
gold_standard_vector = as.vector(unlist(lapply(golub["Class"], map_strings_to_numbers)))
gold_standard_vector

[1] 1 2 2 1 1 2 1 1 2 2 2 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3
[39] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3

Three types of classes within the dataset: **B**, **T** and **M**.

# Clest <a class="anchor" id="clest"></a>
Execution of Clest over the previous dataset.

In [23]:
dataset_matrix = as.matrix(dataset[, -1])
clest_result = clusterAnalysis::clest(dataset_matrix, maxK=10, pmax=0.05, dmin=0.05, B=20,B0=20, clusterAlg=myPam, rho=2/3, similarity=dendextend::FM_index_R)

[1] "Original dataset"
[1] "Number of clusters:  2"
[1] "Number of clusters:  3"
[1] "Number of clusters:  4"
[1] "Number of clusters:  5"
[1] "Number of clusters:  6"
[1] "Number of clusters:  7"
[1] "Number of clusters:  8"
[1] "Number of clusters:  9"
[1] "Number of clusters:  10"

[1] "Reference dataset number 1"
[1] "Number of clusters:  2"
[1] "Number of clusters:  3"
[1] "Number of clusters:  4"
[1] "Number of clusters:  5"
[1] "Number of clusters:  6"
[1] "Number of clusters:  7"
[1] "Number of clusters:  8"
[1] "Number of clusters:  9"
[1] "Number of clusters:  10"

[1] "Reference dataset number 2"
[1] "Number of clusters:  2"
[1] "Number of clusters:  3"
[1] "Number of clusters:  4"
[1] "Number of clusters:  5"
[1] "Number of clusters:  6"
[1] "Number of clusters:  7"
[1] "Number of clusters:  8"
[1] "Number of clusters:  9"
[1] "Number of clusters:  10"

[1] "Reference dataset number 3"
[1] "Number of clusters:  2"
[1] "Number of clusters:  3"
[1] "Number of clusters:  4"
[1

In [25]:
# Golub
pa = cluster::pam(dataset_matrix, 3)
predicted = pa$clustering

In [26]:
sens = MLmetrics::Sensitivity(y_pred = predicted, y_true = gold_standard_vector)
sens = format(round(sens*100, 2), nsmall = 2)
print(paste0("Sensitivity: ", sens, "%"))

[1] "Sensitivity: 97.37%"


In [27]:
cer = CER(predicted, gold_standard_vector)
cer = format(round(cer*100, 2), nsmall = 2)
print(paste0("CER: ", cer, "%"))

[1] "CER: 3.76%"
